In [1]:
!pip install groq python-dotenv numpy tqdm datasets


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.13 -m pip install --upgrade pip


In [2]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

/Users/leeju1_/YBIGTA/[7회차] LLM 과제/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [4]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with any math problems you'd like to solve. Whether you're struggling with a tricky equation or need help with a homework assignment, I'm here to assist you. What kind of math problem would you like to tackle today?


#### GSM8K 데이터셋 확인해보기

In [5]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [6]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [7]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [8]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [9]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [10]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:02<00:02,  1.79it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:06<00:00,  1.64it/s]

Progress: [10/10]
Current Acc.: [30.00%]


In [11]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    print(f"\n[INFO] Running {shot}-shot direct prompting benchmark...\n")
    
    # 1. 프롬프트 구성
    PROMPT = construct_direct_prompt(shot)
    
    # 2. 벤치마크 테스트 수행 (기존 run_benchmark_test 함수 사용)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=False,
        num_samples=50
    )
    
    # 3. 결과 저장
    filename = f"direct_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)
    print(f"[INFO] Saved result to {filename}")


[INFO] Running 0-shot direct prompting benchmark...



 10%|█         | 5/50 [00:03<00:27,  1.63it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:06<00:25,  1.58it/s]

Progress: [10/50]
Current Acc.: [20.00%]


...

Progress: [40/50]
Current Acc.: [15.00%]


 90%|█████████ | 45/50 [02:04<00:13,  2.80s/it]

Progress: [45/50]
Current Acc.: [15.56%]


100%|██████████| 50/50 [02:18<00:00,  2.77s/it]

Progress: [50/50]
Current Acc.: [18.00%]
[INFO] Saved result to direct_prompting_5.txt


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [12]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = (
        "Let's solve some math word problems step-by-step.\n"
        "Please generate the full reasoning and finish with the final answer after '####'.\n"
    )

    for i in range(num_examples):
        #TODO: CoT example을 만들어주세요!
        idx = sampled_indices[i]
        question = train_dataset['question'][idx]
        answer = train_dataset['answer'][idx]
        
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{question}\n"
        prompt += f"Answer:\n{answer}\n" 

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [13]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    print(f"\n[INFO] Running {shot}-shot CoT prompting benchmark...\n")
    
    # 1. 프롬프트 구성
    PROMPT = construct_CoT_prompt(shot)
    
    # 2. 벤치마크 테스트 수행 (기존 run_benchmark_test 함수 사용)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=False,
        num_samples=50
    )
    
    # 3. 결과 저장
    filename = f"CoT_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)
    print(f"[INFO] Saved result to {filename}")


[INFO] Running 0-shot CoT prompting benchmark...



 10%|█         | 5/50 [00:02<00:22,  2.02it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:05<00:21,  1.88it/s]

Progress: [10/50]
Current Acc.: [60.00%]


...

Progress: [40/50]
Current Acc.: [72.50%]


 90%|█████████ | 45/50 [09:43<01:06, 13.22s/it]

Progress: [45/50]
Current Acc.: [71.11%]


100%|██████████| 50/50 [10:51<00:00, 13.03s/it]

Progress: [50/50]
Current Acc.: [72.00%]
[INFO] Saved result to CoT_prompting_5.txt


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [23]:
def construct_my_prompt(shot: int) -> str:
    base_instruction = (
        "You are a world-renowned elementary school math tutor who explains clearly and step-by-step.\n"
        "Solve the following math word problem with step-by-step reasoning.\n"
        "Do all calculations inline using << >>.\n"
        "Always finish with the final answer on a new line like this: '#### <final answer>'.\n"
        "Think carefully and double-check each step.\n"
    )

    if shot == 0:
        return (
            base_instruction
            + "\n[Question]\n{question}\n"
            + "[Answer]"
        )

    # 샷 > 0인 경우 few-shot 예시 포함
    prompt = base_instruction

    filtered = [
        i for i in range(len(gsm8k_train))
        if "####" in gsm8k_train["answer"][i]
        and "<<" in gsm8k_train["answer"][i]
        and 100 < len(gsm8k_train["answer"][i]) < 350
        and gsm8k_train["answer"][i].count("\n") >= 2
    ]

    sampled_indices = random.sample(filtered, shot)

    for i, idx in enumerate(sampled_indices):
        q = gsm8k_train["question"][idx].strip()
        a = gsm8k_train["answer"][idx].strip()

        prompt += f"\n\n[Example {i+1}]\n[Question]\n{q}\n[Answer]\n{a}"

    prompt += "\n\nNow solve the following problem.\n[Question]\n{question}\n[Answer]"

    return prompt

In [24]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    print(f"\n[INFO] Running {shot}-shot my prompting benchmark...\n")
    
    # 1. 프롬프트 구성
    PROMPT = construct_my_prompt(shot)
    
    # 2. 벤치마크 테스트 수행 (기존 run_benchmark_test 함수 사용)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=False,
        num_samples=50
    )
    
    # 3. 결과 저장
    filename = f"My_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)
    print(f"[INFO] Saved result to {filename}")


[INFO] Running 0-shot my prompting benchmark...



 10%|█         | 5/50 [00:04<00:37,  1.19it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:09<00:39,  1.03it/s]

Progress: [10/50]
Current Acc.: [50.00%]


...

Progress: [40/50]
Current Acc.: [70.00%]


 90%|█████████ | 45/50 [07:23<00:48,  9.75s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [08:14<00:00,  9.89s/it]

Progress: [50/50]
Current Acc.: [74.00%]
[INFO] Saved result to My_prompting_5.txt


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!